In [1]:
from pathlib import Path
from IPython.display import HTML, display
css = Path("../../../css/rtl.css").read_text(encoding="utf-8")
display(HTML(f"<style>{css}</style>"))


# فصل ۲ — مبانی داده و پیش‌پردازش
## درس ۱۲: بهداشتِ Train/Validation/Test (برش‌های زمانی، برش‌های گروهی، نشتِ موجودیت)

این درس درباره واقعی‌کردن ارزیابی است. حتی بهترین مدل هم اگر ارزیابی عملکرد را بیش‌ازحد واقعی نشان دهد، تصمیم‌گیری را خراب می‌کند.

در این درس، برش زمان‌محور، برش گروهی/موجودیتی، فورنسیکِ نشت، و الگوهای امن پیش‌پردازش را تمرین می‌کنید. کدها برای جریان‌های کاری رایج در داده‌های جدولی طراحی شده‌اند.


### اهداف یادگیری

در پایان این درس باید بتوانید:

1. توضیح دهید چرا ارزیابی به استقلال تقریبی بین آموزش و آزمون نیاز دارد.
2. تشخیص دهید چه زمانی برش تصادفی قابل قبول است و چه زمانی گمراه‌کننده است.
3. برش زمانی، اعتبارسنجی پنجره‌ای/گسترشی و گَپ زمانی را پیاده‌سازی کنید.
4. برش گروهی/موجودیتی و cross-validation گروه‌محور را پیاده‌سازی کنید.
5. هم‌پوشانی موجودیت، نزدیک-به-تکراری‌ها و ویژگی‌های پس از رخداد را تشخیص دهید.
6. با پایپلاین‌ها از نشت پیش‌پردازش جلوگیری کنید.
7. انتخاب مدل را از آزمون نهایی جدا کنید (نشت اعتبارسنجی نداشته باشید).


### ایده اصلی: مجموعه آزمون دقیقاً چه چیزی را تخمین می‌زند؟

اگر توزیع استقرار $\mathcal{P}$ و تابع هزینه $\ell(\cdot)$ باشد، ریسک تعمیم:

$$R(f) = \mathbb{E}_{(X,Y) \sim \mathcal{P}}[\ell(f(X), Y)].$$

مجموعه آزمون فقط زمانی مفید است که نمونه‌ای تقریباً i.i.d. از $\mathcal{P}$ *در زمان و مقیاس استقرار* باشد.
اگر برش این تقریب را نقض کند (به علت وابستگی زمانی، موجودیت‌های تکراری یا نشت)، تخمین آزمون خوش‌بینانه می‌شود.

به‌صورت فشرده: اگر $\mathcal{D}_{\text{train}}$ و $\mathcal{D}_{\text{test}}$ مستقل نباشند، عموماً:

$$\mathbb{E}[\widehat{R}_{\text{test}}(f)] \ne R(f),$$

زیرا رویه آموزش می‌تواند از هم‌بستگی بین دو مجموعه استفاده کند.


### طبقه‌بندی انواع نشت

نشت یعنی هر مسیری که اطلاعاتِ غیرقابل‌دسترس در زمان پیش‌بینی، بر آموزش یا ارزیابی اثر بگذارد.

- **نشت زمانی:** یادگیری از آینده (مستقیم یا از طریق آمارهایی که آینده را شامل می‌شوند).
- **نشت گروه/موجودیت:** یک موجودیت هم در train و هم در test دیده می‌شود.
- **نشت هدف:** ویژگی‌ها جانشین مستقیم/غیرمستقیم برچسب هستند (پس از رخداد).
- **نشت اعتبارسنجی:** تنظیم‌های مدل با نگاه‌های مکرر به test.

قانون عملی: لحظه/رویداد پیش‌بینی را تعریف کنید و هر ویژگی‌ای را که در آن لحظه نامعلوم است حذف کنید.


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import (
    train_test_split, TimeSeriesSplit, GroupShuffleSplit, GroupKFold, KFold
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import accuracy_score, roc_auc_score, r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 160)


## ۱) یک پروتکل ارزیابی تمیز

یک پروتکل قابل اتکا برای اغلب پروژه‌های جدولی:

1. **قرارداد پیش‌بینی را مشخص کنید**: چه چیزی، چه زمانی، و با چه ورودی‌هایی پیش‌بینی می‌شود.
2. **برش را طراحی کنید** تا بازتاب استقرار باشد (تصادفی، زمانی، گروهی یا ترکیبی).
3. **test را قفل کنید**: برای تصمیم‌های تکراری از آن استفاده نکنید.
4. **روی داده آموزش** از اعتبارسنجی/‏CV برای انتخاب مدل استفاده کنید.
5. در صورت امکان **عدم قطعیت** را با تغییرات foldها گزارش کنید.

برش سه‌گانه:

$$\mathcal{D} = \mathcal{D}_{\text{train}} \cup \mathcal{D}_{\text{val}} \cup \mathcal{D}_{\text{test}},\quad
\mathcal{D}_{\text{train}} \cap \mathcal{D}_{\text{val}} = \varnothing,\quad
\mathcal{D}_{\text{train}} \cap \mathcal{D}_{\text{test}} = \varnothing,\quad
\mathcal{D}_{\text{val}} \cap \mathcal{D}_{\text{test}} = \varnothing.$$

نکته کلیدی این است که «جدا بودن» باید در ساختار وابستگی هم رعایت شود: زمان نباید به عقب برگردد و موجودیت‌ها در حالتِ تعمیم به موجودیت‌های جدید نباید هم‌پوشانی داشته باشند.


### چه زمانی برش تصادفی قابل قبول است؟

برش تصادفی معمولاً وقتی قابل قبول است که:

- ردیف‌ها تقریباً i.i.d. باشند (ترتیب زمانی معنادار و موجودیت‌های تکراری وجود نداشته باشد).
- توزیع استقرار پایدار باشد (drift شدید نباشد).
- مدل روی داده‌ای مشابه داده جمع‌آوری‌شده استفاده شود.

اگر یکی از موارد زیر برقرار باشد، برش تصادفی پرریسک می‌شود:

- چند ردیف برای هر موجودیت.
- وابستگی زمانی یا فصل‌مندی.
- تغییرات عملیاتی (سیاست‌ها، محصول، ارتقای حسگر).
- ویژگی‌های تجمیعی روی پنجره‌های زمانی.

در این شرایط، معمولاً به برش زمان‌محور یا موجودیت‌محور نیاز دارید.


## ۲) برش‌های زمانی

برش زمانی زمانی لازم است که مسئله آینده‌نگر باشد یا فرآیند تولید داده در طول زمان تغییر کند.

یک holdout زمان‌محور:

$$\text{Train} = \{t \le t_0\},\quad \text{Test} = \{t > t_0\}.$$

اگر برچسب با تأخیر مشاهده می‌شود یا ویژگی‌ها روی پنجره‌های زمانی ساخته می‌شوند، یک **گَپ** زمانی هم در نظر بگیرید.


### مثال A: مقایسه برش تصادفی و برش زمانی روی شکایات مصرف‌کنندگان

دیتاست: `ConsumerComplaints.csv` با `Date Received`.

وظیفه: پیش‌بینی اینکه آیا پاسخ‌دهی به‌موقع بوده است (`Timely Response`).
برش تصادفیِ طبقه‌بندی‌شده را با برش رو به آینده مقایسه می‌کنیم.


In [4]:
complaints_path = "../../../Datasets/Clustering/ConsumerComplaints.csv"
df = pd.read_csv(complaints_path, low_memory=False)
df.head()

,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed,Complaint ID
0,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,Yes,No,468882
1,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,Yes,No,468889
2,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,Yes,No,468879
3,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,468949
4,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Franklin Credit Management,CT,6106,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,475823


In [5]:
# Basic cleanup for the demo
df = df.copy()
df['Date Received'] = pd.to_datetime(df['Date Received'], errors='coerce')
df = df.dropna(subset=['Date Received', 'Timely Response'])

y = (df['Timely Response'].astype(str).str.strip().str.lower() == 'yes').astype(int)
feature_cols = ['Product Name', 'Sub Product', 'Issue', 'Sub Issue', 'Company', 'State Name', 'Submitted via']
X = df[feature_cols]

print('Rows:', len(df))
print('Positive rate:', float(y.mean()))
print('Date range:', df['Date Received'].min().date(), 'to', df['Date Received'].max().date())

Rows: 65499
Positive rate: 0.9772210262752103
Date range: 2013-07-22 to 2015-09-02


In [6]:
# پاک‌سازی اولیه برای دمو
df = df.copy()
df['Date Received'] = pd.to_datetime(df['Date Received'], errors='coerce')
df = df.dropna(subset=['Date Received', 'Timely Response'])

y = (df['Timely Response'].astype(str).str.strip().str.lower() == 'yes').astype(int)
feature_cols = ['Product Name', 'Sub Product', 'Issue', 'Sub Issue', 'Company', 'State Name', 'Submitted via']
X = df[feature_cols]

print('تعداد ردیف‌ها:', len(df))
print('نرخ مثبت:', float(y.mean()))
print('بازه تاریخ:', df['Date Received'].min().date(), 'تا', df['Date Received'].max().date())

تعداد ردیف‌ها: 65499
نرخ مثبت: 0.9772210262752103
بازه تاریخ: 2013-07-22 تا 2015-09-02


In [8]:
categorical_features = feature_cols
preprocess = ColumnTransformer(
    transformers=[
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ],
    remainder='drop'
)

clf = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', LogisticRegression(max_iter=200))
])

In [9]:
# (1) Random stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
clf.fit(X_train, y_train)

proba = clf.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

acc_random = accuracy_score(y_test, pred)
auc_random = roc_auc_score(y_test, proba)
print('Random split accuracy:', round(acc_random, 4))
print('Random split ROC-AUC  :', round(auc_random, 4))

Random split accuracy: 0.9781
Random split ROC-AUC  : 0.9154


In [10]:
# (۱) برش تصادفیِ طبقه‌بندی‌شده
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
clf.fit(X_train, y_train)

proba = clf.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

acc_random = accuracy_score(y_test, pred)
auc_random = roc_auc_score(y_test, proba)
print('دقت برش تصادفی:', round(acc_random, 4))
print('ROC-AUC برش تصادفی:', round(auc_random, 4))

دقت برش تصادفی: 0.9781
ROC-AUC برش تصادفی: 0.9154


In [11]:
# (۲) برش زمان‌محور: آموزش روی ۸۰٪ اول و آزمون روی ۲۰٪ آخر
df_sorted = df.sort_values('Date Received')
X_sorted = df_sorted[feature_cols]
y_sorted = (df_sorted['Timely Response'].astype(str).str.strip().str.lower() == 'yes').astype(int)

cut = int(0.8 * len(df_sorted))
X_train_t, X_test_t = X_sorted.iloc[:cut], X_sorted.iloc[cut:]
y_train_t, y_test_t = y_sorted.iloc[:cut], y_sorted.iloc[cut:]

clf.fit(X_train_t, y_train_t)
proba_t = clf.predict_proba(X_test_t)[:, 1]
pred_t = (proba_t >= 0.5).astype(int)

acc_time = accuracy_score(y_test_t, pred_t)
auc_time = roc_auc_score(y_test_t, proba_t)
print('دقت برش زمانی:', round(acc_time, 4))
print('ROC-AUC برش زمانی:', round(auc_time, 4))
print('تاریخ پایان train:', df_sorted['Date Received'].iloc[cut-1].date())
print('تاریخ شروع test :', df_sorted['Date Received'].iloc[cut].date())

دقت برش زمانی: 0.9705
ROC-AUC برش زمانی: 0.8955
تاریخ پایان train: 2015-01-20
تاریخ شروع test : 2015-01-20


### تفسیر اختلاف

اگر امتیاز برش تصادفی بالاتر از برش زمانی باشد، معمولاً «خبر بد» نیست؛ بلکه نشان می‌دهد آینده سخت‌تر از یک عکسِ تصادفیِ درهم‌ریخته است.

علت‌های رایج:

- تغییر توزیع (محصول جدید، تغییر سیاست).
- هم‌بستگی زمانی (تاریخ‌های نزدیک زمینه مشترک دارند).
- تغییر نرخ پایه.

اگر سیستم باید روی دوره‌های آینده پیش‌بینی کند، برش رو به آینده ارزیابی مرتبط است.


### اعتبارسنجی پنجره‌ای/گسترشی با `TimeSeriesSplit`

برای کاهش نویز یک holdout تکی، از rolling validation استفاده کنید.

در پنجره گسترشی:

$$\text{Train}_k = \{t \le t_k\},\quad \text{Test}_k = \{t_k < t \le t_{k+1}\}.$$

این دقیقاً شبیه واقعیت عملی است که تا یک تاریخ آموزش می‌دهید.


In [12]:
tscv = TimeSeriesSplit(n_splits=5)
X_ts = X_sorted.reset_index(drop=True)
y_ts = y_sorted.reset_index(drop=True)

aucs = []
for fold, (tr, te) in enumerate(tscv.split(X_ts), start=1):
    clf.fit(X_ts.iloc[tr], y_ts.iloc[tr])
    proba = clf.predict_proba(X_ts.iloc[te])[:, 1]
    auc = roc_auc_score(y_ts.iloc[te], proba)
    aucs.append(auc)
    train_end = df_sorted.iloc[tr[-1]]['Date Received'].date()
    test_start = df_sorted.iloc[te[0]]['Date Received'].date()
    test_end = df_sorted.iloc[te[-1]]['Date Received'].date()
    print(f'Fold {fold}: AUC={auc:.4f} | پایان train={train_end} | test={test_start}..{test_end}')

print('میانگین AUC:', float(np.mean(aucs)))
print('انحراف معیار AUC:', float(np.std(aucs)))

Fold 1: AUC=0.8265 | پایان train=2013-12-08 | test=2013-12-08..2014-03-25
Fold 2: AUC=0.8235 | پایان train=2014-03-25 | test=2014-03-25..2014-07-08
Fold 3: AUC=0.8747 | پایان train=2014-07-08 | test=2014-07-08..2014-10-20
Fold 4: AUC=0.8834 | پایان train=2014-10-20 | test=2014-10-20..2015-02-09
Fold 5: AUC=0.8959 | پایان train=2015-02-09 | test=2015-02-09..2015-09-02
میانگین AUC: 0.8607962707647356
انحراف معیار AUC: 0.029990864243419388


### محافظ‌های زمانی: گَپ و تأخیر برچسب

اگر برچسب با تأخیر می‌آید یا ویژگی‌ها روی پنجره‌های زمانی ساخته می‌شوند، ممکن است اطلاعات پنجره برچسب وارد ویژگی شود.

یک محافظ ساده گَپ $g$ است:

$$\text{Train} = \{t \le t_0\},\quad \text{Gap} = (t_0, t_0 + g],\quad \text{Test} = \{t > t_0 + g\}.$$

گَپ را متناسب با بیشترین افق نگاه‌به‌آینده در تعریف ویژگی‌ها انتخاب کنید.


## ۳) برش‌های گروهی و موجودیتی

برش گروهی وابستگی ناشی از موجودیت‌های تکراری یا زمینه مشترک را کنترل می‌کند.

دو سؤال عملیِ استقرار:

- **تعمیم به موجودیت‌های جدید؟** → train/test باید بر اساس شناسه جدا باشند.
- **پیش‌بینی آینده برای موجودیت‌های موجود؟** → برش زمانی درون هر موجودیت (و شاید گَپ زمانی).

برش باید دقیقاً مطابق سناریوی استقرار باشد.


### یک استدلال کوتاه برای سوگیری

فرض کنید $G$ شناسه موجودیت باشد و یک اثر پنهان $\alpha_G$ وجود داشته باشد. مدل ساده:

$$Y = \beta^\top X + \alpha_G + \epsilon.$$

اگر همان $G$ها در train و test مشترک باشند، مدل می‌تواند $\alpha_G$ را از داده‌های آموزش حدس بزند و پیش‌بینی روی test ساده‌تر می‌شود.
این باعث می‌شود امتیاز آزمون به سناریوی «موجودیت دیده‌شده» سوگیر شود.

اگر هدف عملکرد روی موجودیت‌های جدید است، باید هم‌پوشانی $G$ را صفر کنید.


### مثال B: نشت در سطح میزبان در listings

دیتاست: `listings.csv`. موجودیت: `host_id`.
وظیفه: پیش‌بینی `room_type`.


In [14]:
listings_path = "../../../Datasets/Regression/listings.csv"
ldf = pd.read_csv(listings_path, low_memory=False)
ldf.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56861,-0.11270,Private room,57.0,1,51,2025-02-09,0.29,3,344,10,NaN
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.48780,-0.16813,Entire home/apt,NaN,4,96,2024-04-28,0.52,1,11,2,NaN
2,17402,Very Central Modern 3-Bed/2 Bath By Oxford St W1,67564,Liz,NaN,Westminster,51.52195,-0.14094,Entire home/apt,510.0,3,56,2024-02-19,0.33,5,293,0,NaN
3,24328,Battersea live/work artist house,41759,Joe,NaN,Wandsworth,51.47072,-0.16266,Entire home/apt,213.0,90,94,2022-07-19,0.54,1,194,0,NaN
4,31036,Bright compact 1 Bedroom Apartment Brick Lane,133271,Hendryks,NaN,Tower Hamlets,51.52425,-0.06997,Entire home/apt,100.0,2,126,2025-02-20,0.70,8,353,3,NaN


In [15]:
ldf = ldf.copy()
ldf = ldf.dropna(subset=['host_id', 'room_type'])

features = ['neighbourhood', 'latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']
for col in ['price', 'reviews_per_month']:
    ldf[col] = pd.to_numeric(ldf[col], errors='coerce')

X2 = ldf[features]
y2 = ldf['room_type'].astype(str)
groups = ldf['host_id'].astype(str)

print('تعداد ردیف‌ها:', len(ldf))
print('تعداد میزبان یکتا:', groups.nunique())
print('فراوانی کلاس‌ها (چند مورد اول):')
print(y2.value_counts().head())

تعداد ردیف‌ها: 94559
تعداد میزبان یکتا: 55395
فراوانی کلاس‌ها (چند مورد اول):
room_type
Entire home/apt    60750
Private room       33487
Shared room          164
Hotel room           158
Name: count, dtype: int64


In [16]:
ldf = ldf.copy()
ldf = ldf.dropna(subset=['host_id', 'room_type'])

features = ['neighbourhood', 'latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']
for col in ['price', 'reviews_per_month']:
    ldf[col] = pd.to_numeric(ldf[col], errors='coerce')

X2 = ldf[features]
y2 = ldf['room_type'].astype(str)
groups = ldf['host_id'].astype(str)

print('Rows:', len(ldf))
print('Unique hosts:', groups.nunique())
print('Class counts (top):')
print(y2.value_counts().head())

Rows: 94559
Unique hosts: 55395
Class counts (top):
room_type
Entire home/apt    60750
Private room       33487
Shared room          164
Hotel room           158
Name: count, dtype: int64


In [19]:
num_features = ['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']
cat_features = ['neighbourhood']

preprocess2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                               ('scaler', StandardScaler())]), num_features),
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                               ('onehot', OneHotEncoder(handle_unknown='ignore'))]), cat_features)
    ],
    remainder='drop'
)

clf2 = Pipeline(steps=[
    ('preprocess', preprocess2),
    ('model', LogisticRegression(max_iter=300))
])

In [20]:
# برش تصادفی
X_tr, X_te, y_tr, y_te, g_tr, g_te = train_test_split(
    X2, y2, groups, test_size=0.2, random_state=42, stratify=y2
)
clf2.fit(X_tr, y_tr)
pred = clf2.predict(X_te)
acc = accuracy_score(y_te, pred)

shared_hosts = len(set(g_tr) & set(g_te))
print('دقت برش تصادفی:', round(acc, 4))
print('میزبان‌های مشترک train/test:', shared_hosts)
print('میزبان‌های train:', len(set(g_tr)), '| میزبان‌های test:', len(set(g_te)))

دقت برش تصادفی: 0.7474
میزبان‌های مشترک train/test: 5126
میزبان‌های train: 46337 | میزبان‌های test: 14184


In [21]:
# برش گروهی (میزبان‌ها جدا)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr_idx, te_idx = next(gss.split(X2, y2, groups=groups))

X_trg, X_teg = X2.iloc[tr_idx], X2.iloc[te_idx]
y_trg, y_teg = y2.iloc[tr_idx], y2.iloc[te_idx]
g_trg, g_teg = groups.iloc[tr_idx], groups.iloc[te_idx]

clf2.fit(X_trg, y_trg)
pred_g = clf2.predict(X_teg)
acc_g = accuracy_score(y_teg, pred_g)

shared_hosts_g = len(set(g_trg) & set(g_teg))
print('دقت برش گروهی:', round(acc_g, 4))
print('میزبان‌های مشترک train/test:', shared_hosts_g)
print('میزبان‌های train:', len(set(g_trg)), '| میزبان‌های test:', len(set(g_teg)))

دقت برش گروهی: 0.7529
میزبان‌های مشترک train/test: 0
میزبان‌های train: 44316 | میزبان‌های test: 11079


### فورنسیک نشت: هم‌پوشانی و نزدیک-به-تکراری‌ها

حتی اگر شناسه موجودیت‌ها جدا باشد، نزدیک-به-تکراری‌ها می‌توانند نشت ایجاد کنند.

یک کنترل ساده این است که روی چند ستون نسبتاً پایدار یک امضا (هش) بسازید و تعداد امضاهای مشترک بین train و test را بشمارید.


In [22]:
def overlap_count(a, b):
    return len(set(a) & set(b))

sig_cols = ['neighbourhood', 'latitude', 'longitude', 'minimum_nights']
sig = X2[sig_cols].copy()
sig['latitude'] = sig['latitude'].round(5)
sig['longitude'] = sig['longitude'].round(5)
signature = pd.util.hash_pandas_object(sig, index=False)

sig_tr = signature.iloc[X_tr.index]
sig_te = signature.iloc[X_te.index]
dup_random = overlap_count(sig_tr, sig_te)

sig_trg = signature.iloc[tr_idx]
sig_teg = signature.iloc[te_idx]
dup_group = overlap_count(sig_trg, sig_teg)

print('امضاهای تکراری تقریبی (برش تصادفی):', dup_random)
print('امضاهای تکراری تقریبی (برش گروهی) :', dup_group)

امضاهای تکراری تقریبی (برش تصادفی): 556
امضاهای تکراری تقریبی (برش گروهی) : 59


### Cross-validation گروه‌محور با `GroupKFold`

برای چند fold با حفظ جدایی موجودیت‌ها از `GroupKFold` استفاده کنید.
این کار هم تخمین واریانس می‌دهد و هم وابستگی به یک برش دلخواه را کم می‌کند.


In [23]:
gkf = GroupKFold(n_splits=5)
accs = []
for fold, (tr, te) in enumerate(gkf.split(X2, y2, groups=groups), start=1):
    clf2.fit(X2.iloc[tr], y2.iloc[tr])
    pred = clf2.predict(X2.iloc[te])
    acc = accuracy_score(y2.iloc[te], pred)
    accs.append(acc)
    shared = len(set(groups.iloc[tr]) & set(groups.iloc[te]))
    print(f'Fold {fold}: acc={acc:.4f} | میزبان مشترک={shared} | میزبان‌های test={groups.iloc[te].nunique()}')

print('میانگین دقت:', float(np.mean(accs)))
print('انحراف معیار دقت:', float(np.std(accs)))

Fold 1: acc=0.7404 | میزبان مشترک=0 | میزبان‌های test=11078
Fold 2: acc=0.7416 | میزبان مشترک=0 | میزبان‌های test=11079
Fold 3: acc=0.7348 | میزبان مشترک=0 | میزبان‌های test=11079
Fold 4: acc=0.7481 | میزبان مشترک=0 | میزبان‌های test=11080
Fold 5: acc=0.7515 | میزبان مشترک=0 | میزبان‌های test=11079
میانگین دقت: 0.7432714839285025
انحراف معیار دقت: 0.00591220648589423


## ۴) ساختار ترکیبی: زمان + موجودیت (داده‌های پنلی)

بسیاری از دیتاست‌های واقعی **پنلی** هستند: اندازه‌گیری‌های تکراری برای هر موجودیت در طول زمان.

در داده پنلی معمولاً باید بین دو هدف ارزیابی انتخاب کنید:

- **تعمیم به موجودیت‌های دیده‌نشده**
- **پیش‌بینی آینده برای موجودیت‌های دیده‌شده**

این‌ها به برش‌های متفاوت نیاز دارند و عددهای عملکرد سؤالات متفاوتی را پاسخ می‌دهند.


### مثال C: داده پنلی آموزش (`states_all.csv`)

دیتاست: `states_all.csv` با ستون‌های `STATE` و `YEAR`.

وظیفه: پیش‌بینی `AVG_MATH_8_SCORE` از متغیرهای درآمد/هزینه.

سه برش را مقایسه می‌کنیم:

1. برش تصادفی ردیف‌ها (اغلب خوش‌بینانه چون یک STATE در هر دو مجموعه می‌آید).
2. برش گروهی بر اساس `STATE` (ارزیابی روی ایالت‌های جدید).
3. برش زمانی درون هر STATE (پیش‌بینی سال‌های آینده همان ایالت‌ها).


In [24]:
states_path = "../../../Datasets/Regression/states_all.csv"
sdf = pd.read_csv(states_path, low_memory=False)
sdf.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN


In [25]:
sdf = sdf.copy()
sdf['YEAR'] = pd.to_numeric(sdf['YEAR'], errors='coerce')
sdf = sdf.dropna(subset=['STATE', 'YEAR', 'AVG_MATH_8_SCORE'])

target = 'AVG_MATH_8_SCORE'
group_col = 'STATE'

num_cols = [
    'ENROLL', 'TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
    'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE'
]
X4 = sdf[num_cols].apply(pd.to_numeric, errors='coerce')
y4 = pd.to_numeric(sdf[target], errors='coerce')
g4 = sdf[group_col].astype(str)
t4 = sdf['YEAR'].astype(int)

mask = y4.notna()
X4, y4, g4, t4 = X4.loc[mask], y4.loc[mask], g4.loc[mask], t4.loc[mask]

print('تعداد ردیف‌ها:', len(X4))
print('تعداد ایالت‌ها:', g4.nunique())
print('بازه سال:', int(t4.min()), 'تا', int(t4.max()))

تعداد ردیف‌ها: 602
تعداد ایالت‌ها: 53
بازه سال: 1990 تا 2019


In [26]:
reg_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', Ridge())
])

def eval_regression(y_true, y_pred):
    # RMSE بدون استفاده از پارامتر deprecated
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    r2 = r2_score(y_true, y_pred)
    return rmse, r2


In [27]:
# (۱) برش تصادفی ردیف‌ها
X_tr, X_te, y_tr, y_te, g_tr, g_te, t_tr, t_te = train_test_split(
    X4, y4, g4, t4, test_size=0.2, random_state=42
)
reg_pipe.fit(X_tr, y_tr)
pred = reg_pipe.predict(X_te)
rmse, r2 = eval_regression(y_te, pred)
print('برش تصادفی | RMSE:', round(rmse, 3), '| R^2:', round(r2, 3))
print('ایالت مشترک:', len(set(g_tr) & set(g_te)))
print('سال مشترک  :', len(set(t_tr) & set(t_te)))

برش تصادفی | RMSE: 9.193 | R^2: 0.092
ایالت مشترک: 48
سال مشترک  : 12


In [28]:
# (۲) برش گروهی بر اساس STATE (ایالت‌های جدید)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr_idx, te_idx = next(gss.split(X4, y4, groups=g4))
X_trg, X_teg = X4.iloc[tr_idx], X4.iloc[te_idx]
y_trg, y_teg = y4.iloc[tr_idx], y4.iloc[te_idx]
g_trg, g_teg = g4.iloc[tr_idx], g4.iloc[te_idx]

reg_pipe.fit(X_trg, y_trg)
pred = reg_pipe.predict(X_teg)
rmse, r2 = eval_regression(y_teg, pred)
print('برش گروهی (STATE) | RMSE:', round(rmse, 3), '| R^2:', round(r2, 3))
print('ایالت مشترک:', len(set(g_trg) & set(g_teg)))

برش گروهی (STATE) | RMSE: 9.889 | R^2: 0.164
ایالت مشترک: 0


In [29]:
# (۳) برش زمانی درون STATE: برای هر ایالت ۲۰٪ آخر سال‌ها را test بگیرید
sdf_work = pd.DataFrame({'state': g4.values, 'year': t4.values})
sdf_work['idx'] = np.arange(len(sdf_work))

test_mask = np.zeros(len(sdf_work), dtype=bool)
for state, sub in sdf_work.groupby('state'):
    years = np.sort(sub['year'].unique())
    if len(years) < 5:
        continue
    cut_year = years[int(np.floor(0.8 * len(years)))]
    sub_idx = sub.loc[sub['year'] > cut_year, 'idx'].values
    test_mask[sub_idx] = True

tr_idx = np.where(~test_mask)[0]
te_idx = np.where(test_mask)[0]

X_trt, X_tet = X4.iloc[tr_idx], X4.iloc[te_idx]
y_trt, y_tet = y4.iloc[tr_idx], y4.iloc[te_idx]

reg_pipe.fit(X_trt, y_trt)
pred = reg_pipe.predict(X_tet)
rmse, r2 = eval_regression(y_tet, pred)
print('برش زمانی درون ایالت | RMSE:', round(rmse, 3), '| R^2:', round(r2, 3))

violations = 0
for state, sub in sdf_work.groupby('state'):
    tr_years = t4.iloc[tr_idx][g4.iloc[tr_idx] == state]
    te_years = t4.iloc[te_idx][g4.iloc[te_idx] == state]
    if len(tr_years) and len(te_years) and tr_years.max() >= te_years.min():
        violations += 1
print('تعداد ایالت‌های دارای نقض ترتیب زمانی:', violations)

برش زمانی درون ایالت | RMSE: 8.322 | R^2: -0.604
تعداد ایالت‌های دارای نقض ترتیب زمانی: 0


### خواندن نتایجِ برش پنلی

این سه عدد به سه سؤال متفاوت پاسخ می‌دهند:

- برش تصادفی: «اگر ایالت‌ها و سال‌ها را قاطی کنیم، چقدر خوب پیش‌بینی می‌کنیم؟» (اغلب خوش‌بینانه)
- برش گروهی: «چقدر به ایالت‌های کاملاً جدید تعمیم می‌دهیم؟» (سخت‌تر)
- برش زمانی درون ایالت: «چقدر سال‌های آینده همان ایالت‌ها را پیش‌بینی می‌کنیم؟» (شبیه استقرارِ پایش موجودیت‌های موجود)

برش اشتباه می‌تواند تصمیم محصول را اشتباه کند.


## ۵) بهداشت پیش‌پردازش: fit فقط روی train

گام‌های پیش‌پردازش (جایگذاری، مقیاس‌بندی، کدگذاری) پارامترهایی را از داده تخمین می‌زنند.

مثلاً استانداردسازی:

$$\tilde{x} = \frac{x - \mu}{\sigma},\quad \mu = \frac{1}{n}\sum_{i=1}^n x_i,\quad \sigma^2 = \frac{1}{n}\sum_{i=1}^n (x_i - \mu)^2.$$

اگر $\mu$ و $\sigma$ با ردیف‌های test محاسبه شوند، آموزش به‌طور غیرمستقیم از اطلاعات test استفاده کرده است.
پایپلاین‌ها تضمین می‌کنند تخمین پارامترها داخل برش انجام شود.


### مثال D: نشت مقیاس‌بندی در دیتاست دیابت

وظیفه: پیش‌بینی `classification`.
یک الگوی نشت‌دار را با الگوی صحیح مقایسه می‌کنیم.


In [31]:
diabetes_path = "../../../Datasets/Classification/diabetes.csv"
ddf = pd.read_csv(diabetes_path)
ddf.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification
0,6,148,72,35,0,33.6,0.627,50,Diabetic
1,1,85,66,29,0,26.6,0.351,31,Non-Diabetic
2,8,183,64,0,0,23.3,0.672,32,Diabetic
3,1,89,66,23,94,28.1,0.167,21,Non-Diabetic
4,0,137,40,35,168,43.1,2.288,33,Diabetic


In [32]:
ddf = ddf.copy()
y5 = (ddf['classification'].astype(str).str.strip().str.lower() == 'diabetic').astype(int)
X5 = ddf.drop(columns=['classification'])

X_tr, X_te, y_tr, y_te = train_test_split(X5, y5, test_size=0.25, random_state=42, stratify=y5)
print('Train size:', len(X_tr), '| Test size:', len(X_te))

Train size: 576 | Test size: 192


In [33]:
ddf = pd.read_csv(diabetes_path)
ddf = ddf.copy()
y5 = (ddf['classification'].astype(str).str.strip().str.lower() == 'diabetic').astype(int)
X5 = ddf.drop(columns=['classification'])

X_tr, X_te, y_tr, y_te = train_test_split(X5, y5, test_size=0.25, random_state=42, stratify=y5)
print('اندازه train:', len(X_tr), '| اندازه test:', len(X_te))

اندازه train: 576 | اندازه test: 192


In [34]:
# نشت‌دار: fit روی کل داده
scaler_bad = StandardScaler().fit(X5)
X_tr_bad = scaler_bad.transform(X_tr)
X_te_bad = scaler_bad.transform(X_te)

m_bad = LogisticRegression(max_iter=600)
m_bad.fit(X_tr_bad, y_tr)
auc_bad = roc_auc_score(y_te, m_bad.predict_proba(X_te_bad)[:, 1])
print('ROC-AUC (scaling نشت‌دار):', round(auc_bad, 4))

ROC-AUC (scaling نشت‌دار): 0.832


In [35]:
# صحیح: fit فقط روی train
scaler_ok = StandardScaler().fit(X_tr)
X_tr_ok = scaler_ok.transform(X_tr)
X_te_ok = scaler_ok.transform(X_te)

m_ok = LogisticRegression(max_iter=600)
m_ok.fit(X_tr_ok, y_tr)
auc_ok = roc_auc_score(y_te, m_ok.predict_proba(X_te_ok)[:, 1])
print('ROC-AUC (scaling صحیح):', round(auc_ok, 4))

ROC-AUC (scaling صحیح): 0.832


In [36]:
# بهترین روش: Pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(max_iter=600))])
pipe.fit(X_tr, y_tr)
auc_pipe = roc_auc_score(y_te, pipe.predict_proba(X_te)[:, 1])
print('ROC-AUC (pipeline):', round(auc_pipe, 4))

ROC-AUC (pipeline): 0.832


## ۶) Target encoding: جلوگیری از نشت با cross-fitting

Target encoding برای دسته‌های با کاردینالیتی بالا مفید است، اما از پرخطرترین مسیرهای نشت محسوب می‌شود.

Target encoding ساده:

$$\widehat{m}(c) = \frac{1}{|\{i : C_i=c\}|}\sum_{i:C_i=c} Y_i.$$

اگر ردیف‌های test در $\widehat{m}(c)$ اثر داشته باشند، ارزیابی خوش‌بینانه می‌شود.
Cross-fitting یک encoding شبه خارج-از-نمونه برای ردیف‌های train می‌سازد.


In [37]:
def target_encode_crossfit(train_col: pd.Series, y: pd.Series, n_splits: int = 5, smoothing: float = 20.0, random_state: int = 42):
    """Cross-fitted target encoding for a single categorical column.
    Returns: encoded_train (aligned to train_col index), enc_map (means on full train), global_mean
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    global_mean = float(y.mean())
    enc = pd.Series(index=train_col.index, dtype=float)
    for tr_idx, te_idx in kf.split(train_col):
        tr_c = train_col.iloc[tr_idx]
        tr_y = y.iloc[tr_idx]
        stats = tr_y.groupby(tr_c).agg(['mean', 'count'])
        smooth = (stats['count'] * stats['mean'] + smoothing * global_mean) / (stats['count'] + smoothing)
        te_c = train_col.iloc[te_idx]
        enc.iloc[te_idx] = te_c.map(smooth).fillna(global_mean).astype(float)
    full_stats = y.groupby(train_col).agg(['mean', 'count'])
    full_smooth = (full_stats['count'] * full_stats['mean'] + smoothing * global_mean) / (full_stats['count'] + smoothing)
    return enc, full_smooth, global_mean

def target_encode_apply(col: pd.Series, enc_map: pd.Series, global_mean: float):
    return col.map(enc_map).fillna(global_mean).astype(float)


In [38]:
df_small = df_sorted.tail(60000).copy()
df_small = df_small.dropna(subset=['Company', 'Timely Response'])
y6 = (df_small['Timely Response'].astype(str).str.strip().str.lower() == 'yes').astype(int)
X6 = df_small[['Company', 'State Name', 'Submitted via', 'Product Name']].copy()

X_tr, X_te, y_tr, y_te = train_test_split(X6, y6, test_size=0.2, random_state=42, stratify=y6)
print('ردیف‌های train:', len(X_tr), '| ردیف‌های test:', len(X_te))
print('شرکت‌های یکتا در train:', X_tr['Company'].nunique())

ردیف‌های train: 48000 | ردیف‌های test: 12000
شرکت‌های یکتا در train: 1710


In [39]:
# encoding نشت‌دار (در کار واقعی انجام ندهید)
global_mean_all = float(pd.concat([y_tr, y_te]).mean())
tmp = pd.DataFrame({'Company': pd.concat([X_tr['Company'], X_te['Company']]).values,
                    'y': pd.concat([y_tr, y_te]).values})
means_all = tmp.groupby('Company')['y'].mean()

X_tr_leak = X_tr.copy(); X_te_leak = X_te.copy()
X_tr_leak['Company_te'] = X_tr_leak['Company'].map(means_all).fillna(global_mean_all)
X_te_leak['Company_te'] = X_te_leak['Company'].map(means_all).fillna(global_mean_all)

pre = ColumnTransformer([
    ('num', 'passthrough', ['Company_te']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['State Name', 'Submitted via', 'Product Name']),
])
m = Pipeline([('pre', pre), ('lr', LogisticRegression(max_iter=300))])
m.fit(X_tr_leak[['Company_te', 'State Name', 'Submitted via', 'Product Name']], y_tr)
auc_leaky = roc_auc_score(y_te, m.predict_proba(X_te_leak[['Company_te', 'State Name', 'Submitted via', 'Product Name']])[:, 1])
print('ROC-AUC (encoding نشت‌دار):', round(auc_leaky, 4))

ROC-AUC (encoding نشت‌دار): 0.9417


In [40]:
# encoding امن با cross-fitting
enc_tr, enc_map, gmean = target_encode_crossfit(X_tr['Company'], y_tr, n_splits=5, smoothing=50.0)
X_tr_safe = X_tr.copy(); X_te_safe = X_te.copy()
X_tr_safe['Company_te'] = enc_tr
X_te_safe['Company_te'] = target_encode_apply(X_te_safe['Company'], enc_map, gmean)

m2 = Pipeline([('pre', pre), ('lr', LogisticRegression(max_iter=300))])
m2.fit(X_tr_safe[['Company_te', 'State Name', 'Submitted via', 'Product Name']], y_tr)
auc_safe = roc_auc_score(y_te, m2.predict_proba(X_te_safe[['Company_te', 'State Name', 'Submitted via', 'Product Name']])[:, 1])
print('ROC-AUC (encoding امن):', round(auc_safe, 4))

ROC-AUC (encoding امن): 0.8669


## ۷) نشت اعتبارسنجی و انتخاب مدلِ صادقانه

یک خطای فرایندی رایج این است که هنگام تکرارهای مدل‌سازی، مرتباً به test نگاه کنیم.
این کار test را به validation تبدیل می‌کند و عدد نهایی خوش‌بینانه می‌شود.

الگوی امن‌تر:

1. یک test (زمان‌محور یا گروه‌محور) جدا کنید.
2. روی بقیه داده‌ها، CV/validation برای جست‌وجوی هایپرپارامتر انجام دهید.
3. مدل انتخاب‌شده را روی کل داده غیر-test fit کنید.
4. فقط یک‌بار روی test ارزیابی کنید.

در ادامه یک مثال کوچک روی دیتاست دیابت می‌بینید که در انتخاب پارامترها هرگز از برچسب‌های test استفاده نمی‌کند.


In [41]:
from sklearn.linear_model import LogisticRegression

ddf = pd.read_csv(diabetes_path)
y7 = (ddf['classification'].astype(str).str.strip().str.lower() == 'diabetic').astype(int)
X7 = ddf.drop(columns=['classification'])

X_train, X_test, y_train, y_test = train_test_split(X7, y7, test_size=0.25, random_state=42, stratify=y7)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(max_iter=800))
])

param_grid = {
    'lr__C': [0.05, 0.1, 0.3, 1.0, 3.0, 10.0],
    'lr__penalty': ['l2'],
    'lr__solver': ['lbfgs'],
}

search = GridSearchCV(pipe, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=None)
search.fit(X_train, y_train)
print('بهترین AUC در CV:', round(search.best_score_, 4))
print('بهترین پارامترها:', search.best_params_)

best_model = search.best_estimator_
test_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
print('AUC روی test:', round(test_auc, 4))

بهترین AUC در CV: 0.8249
بهترین پارامترها: {'lr__C': 0.05, 'lr__penalty': 'l2', 'lr__solver': 'lbfgs'}
AUC روی test: 0.8301


## ۸) چک‌لیست عملی

قبل از اعتماد به ارزیابی:

### طراحی برش

1. لحظه پیش‌بینی مشخص است.
2. برش مطابق استقرار است:
   - پیش‌بینی رو به آینده → برش زمانی / rolling CV
   - موجودیت‌های جدید → برش گروهی
   - داده پنلی → برش زمانی درون موجودیت (و شاید گَپ)

### دفاع در برابر نشت

3. نشت پیش‌پردازش ندارید (Pipeline).
4. نشت هدف ندارید (حذف ویژگی‌های پس از رخداد).
5. نشت اعتبارسنجی ندارید (test فقط یک‌بار).

### فورنسیک

6. هم‌پوشانی موجودیت‌ها صفر است وقتی باید باشد.
7. هم‌پوشانی نزدیک-به-تکراری کم است.
8. پایداری عملکرد روی foldها قابل قبول است.


## ۹) تمرین‌ها

1. در دیتاست شکایات، برش زمانی ۷۰/۳۰ و ۹۰/۱۰ را هم امتحان کنید.
2. در listings، هدف را به `availability_365 > 200` تغییر دهید و برش تصادفی را با برش میزبان‌-جدا مقایسه کنید.
3. در دیتاست پنلی (`states_all.csv`)، به‌جای math، `AVG_READING_8_SCORE` را پیش‌بینی کنید.
4. یک گَپ زمانی در برش پنلی اضافه کنید (مثلاً یک سال فاصله بین train و test) و اثر را ببینید.
5. در ConsumerComplaints چند ستون پس از رخداد پیدا کنید که باید حذف شوند.


### نکته کلیدی

طراحی برش بخشی از مدل‌سازی است. برای عملکرد قابل اتکا باید در همان ساختار وابستگی و همان محدودیت‌های اطلاعاتی که در تولید وجود دارد، ارزیابی کنید.
